# CC3001 Otoño 2020 Tarea 4
# Generación de código

Profesores: Sección 1 Patricio Poblete / Sección 2 Nelson Baloian

Plazo: Lunes 8 de junio de 2020 a las 23:59

# Nombre: Vincko Fabres

El objetivo de esta tarea es aprender sobre la manipulación de árboles que representan fórmulas. Hay muchas cosas que uno puede hacer con estos árboles, y en esta tarea nos enfocaremos en particular en algo que los compiladores hacen: la **generación de código** para evaluar fórmulas.

Específicamente, el problema es: dada una fórmula, generar una secuencia de instrucciones elementales que la evalúen. Por ejemplo, dada una fórmula como

```
(a+b)*(c-d)
```

se podría generar código como el siguiente:

```
t1=a+b
t2=c-d
t3=t1*t2
```

Las instrucciones elementales son siempre de la forma ``ti=x op y``, donde ``ti`` es una variable temporal, ``x`` e ``y`` pueden ser variables o números, y ``op`` es un operador. Uno va creando variables ``t1``, ``t2``, ... a medida que las necesita, y la última de ellas es la que almacena el valor final de la fórmula.

Las fórmulas pueden ser arbitrariamente complicadas, como por ejemplo la fórmula

$$
(2-p\,q)\left( \frac{1}{n}+\frac{1}{p+q}\right)
$$

la cual escribimos como

```
(2-p*q)*(1/n+1/(p+q))
```

y cuyo código generado podría ser,

```
t1=p*q
t2=2-t1
t3=1/n
t4=p+q
t5=1/t4
t6=t3+t5
t7=t2*t6
```

El objetivo de esta tarea es que usted implemente una función que, dado un árbol ya construído, lo recorra para generar una secuencia de instrucciones elementales que lo evalúe.

A continuación usted encontrará clases ``Nodoi``, ``Nodoe`` y ``Arbol`` que implementan árboles que representan fórmulas. La clase ``Arbol`` viene con un constructor que recibe un string como parámetro, el cual contiene una fórmula, y este constructor se encarga de transformar la fórmula de string a árbol, de modo que usted no necesita preocuparse de eso. Para simplificar, la fórmula solo podrá contener variables de una sola letra, números de un solo dígito, y sin espacios. Los operadores permitidos son solo suma, resta, multiplicación y división. No hay operador menos unario.

Usted tienen que implementar para la clase ``Arbol`` una función ``codigo`` que al ser invocado genere una secuencia de instrucciones elementales para evaluar la fórmula representada por ese árbol, y la retorne en forma de una lista (de Python).

In [0]:

class Nodoi:
    def __init__(self, izq, info, der):
        self.izq=izq
        self.info=info
        self.der=der
    def postorden(self):
        self.izq.postorden()   
        self.der.postorden()
        print(self.info, end=" ")
    
    def esinterno(self):
        return True

    def esexterno(self):
        return False

    def contarnodosi(self):                 #contador de nodos internos (operaciones)
        n=1
        if self.der.esinterno():
          n+=self.der.contarnodosi()
        if self.izq.esinterno():
          n+=self.izq.contarnodosi()
        return n

    
    def orden(self):                        #genera lista de los elementos del arbol recorridos en postorden
        L=[] 
        L+=self.izq.orden()
        L+=self.der.orden()
        L.append(self.info)
        return L

class Nodoe:
    def __init__(self, info):
        self.info=info
    def postorden(self):
        print(self.info, end=" ")
    def esexterno(self):
		    return True
    def esinterno(self):
        return False

    def orden(self):                        #genera lista de los elementos del arbol recorridos en postorden
        return [self.info]

class Arbol:
    def __init__(self,formula):
        global k
        global s
        s=formula+";" # agregamos una marca de fin de la entrada
        k=0 # indica próximo caracter por procesar
        # definimos funciones para analizar la fórmula
        def expresion(): # retorna puntero a la raíz de un árbol que representa a la fórmula s
            global k
            global s
            a=factor()
            while s[k]=="+" or s[k]=="-":
                op=s[k]
                k+=1
                b=factor()
                a=Nodoi(a,op,b)
            return a
        def factor():
            global k
            global s
            a=termino()
            while s[k]=="*" or s[k]=="/":
                op=s[k]
                k+=1
                b=termino()
                a=Nodoi(a,op,b)
            return a
        def termino(): # posible constante, variable o formula parentizada
            global k
            global s
            if s[k].isalpha() or s[k].isdigit():
                a=Nodoe(s[k])
                k+=1
                return a
            if s[k]=="(": # fórmula parentizada
                k+=1
                a=expresion()
                if s[k]!=")":
                    print("Error: Falta cierra paréntesis: "+formula[k:])
                    assert False
                k+=1
                return a
            print("Error: Falta variable, número o abre paréntesis: "+formula[k:])
            assert False
              
        a=expresion()
        if s[k]!=";":
            print("Error: Basura al final de la fórmula: "+formula[k:])
            assert False                
        self.raiz=a
            
    def postorden(self):
        print("Postorden:", end=" ")
        self.raiz.postorden()
        print()
    
    def orden(self):                  #recorre el arbol en postorden, agrega los elementos a una lista y la retorna
        return self.raiz.orden()

    def contarnodosi(self):                   #cuenta la cantidad de nodos internos(numero de operaciones)
        return self.raiz.contarnodosi()


    def codigo(self):
        Lista=[]             #lista a retornar
        L1=self.raiz.orden() #lista con elementos ordenados en postorden
        i=0                  # recorre elementos de la lista en postorden
        t=1                  #generador de operaciones ti
        if len(L1)==1:                        #caso base de solo una hoja
          return ['t1='+L1[0]]
        operaciones=self.raiz.contarnodosi()  # numero de operaciones
        if operaciones==1:                    #caso base de solo un nodo interno
          return ['t1='+L1[0]+L1[2]+L1[1]]
        while t < operaciones+1:                # el numero de operaciones es igual al numero de ti
          if L1[i]=='+' or L1[i]=='-' or L1[i]=='*' or L1[i]=='/':#revisa si el elemento de la lista es un operador, si lo es se aplica a los 2 numeros anteriores
            operando1=L1[i-2]
            operando2=L1[i-1]
            op=L1[i]
            Lista.append('t'+str(t)+'='+operando1+op+operando2)   #agrega operacion ti a la lista que se retornará
            L1[i-2]='t'+str(t)                                    #ti reemplaza la operacion realizada en la lista de elementos en postorden
            L1.pop(i-1)
            L1.pop(i-1)
            i=0
            t+=1                                                 
          i+=1
        return Lista

A continuación probamos esta funcionalidad con varias fórmulas de ejemplo. Para ver si el árbol fue construido correctamente, lo recorremos en postorden para imprimirlo en notación polaca. **Importante**: La notación polaca no tiene nada que ver con lo que ustedes tienen que hacer, es solo una manera de poder imprimir el árbol generado para chequear que esté bien construido.

Igualmente para corroborar la funcionalidad de la funcion orden se compara el resultado de la lista que genera con la funcion postorden que imprime como salida cada nodo, una vez verificado se implementa en la funcion código

La función código genera 2 listas, una vacía, en la cual se insertarán las operaciones para cada variable temporal ti que posteriormente será retornada y la lista que posee todos los elemntos del árbol recorridos en postorden.
Esta última será recorrida elemento a elemento, cuando el elemento sea un operador se utilizarán los 2 operandos anteriores y el operador, los que se igualarán a una variable temporal ti, la que reemplazará estos 3 términos en la lista, posteriormente se vuelve a recorrer la lista desde el inicio hasta que el número de variables ti sea igual al numero de operaciones

In [79]:
Arbol("(a+b)*(c-d)").postorden()

Postorden: a b + c d - * 


In [80]:
Arbol("(a+b)*(c-d)").orden()

['a', 'b', '+', 'c', 'd', '-', '*']

In [81]:
Arbol("a+b-c*d/e*f").postorden()

Postorden: a b + c d * e / f * - 


In [82]:
Arbol("a+b-c*d/e*f").orden()

['a', 'b', '+', 'c', 'd', '*', 'e', '/', 'f', '*', '-']

In [83]:
Arbol("(2-p*q)*(1/n+1/(p+q))").postorden()

Postorden: 2 p q * - 1 n / 1 p q + / + * 


In [84]:
Arbol("(2-p*q)*(1/n+1/(p+q))").orden()

['2', 'p', 'q', '*', '-', '1', 'n', '/', '1', 'p', 'q', '+', '/', '+', '*']

## Resultados

En esta sección probamos la generación de código que usted tiene que escribir. Mientras usted no sustituya la función ``codigo`` provisoria que viene en la definición de la clase, va a aparecer la lista que generamos ahí. Primero vamos a definir una función ``probar`` que genera el código a partir de una fórmula dada e imprime el resultado:

In [0]:
def probar(formula):
    a=Arbol(formula)
    for x in a.codigo():
        print(x)

A continuación probamos con varias fórmulas. Además de las fórmulas dadas, usted tiene que agregar tres ejemplos adicionales.

In [86]:
probar("a+1")

t1=a+1


In [87]:
probar("(a+b)*(c-d)")

t1=a+b
t2=c-d
t3=t1*t2


In [88]:
probar("(2-p*q)*(1/n+1/(p+q))")

t1=p*q
t2=2-t1
t3=1/n
t4=p+q
t5=1/t4
t6=t3+t5
t7=t2*t6


In [89]:
probar("((((a+b)+c)+d)+e)")

t1=a+b
t2=t1+c
t3=t2+d
t4=t3+e


In [90]:
probar("(a+(b+(c+(d+e))))")

t1=d+e
t2=c+t1
t3=b+t2
t4=a+t3


In [91]:
probar("a")

t1=a


In [92]:
probar("a/c+c")

t1=a/c
t2=t1+c


In [93]:
probar("c+d-b+a")

t1=c+d
t2=t1-b
t3=t2+a


In [95]:
probar("c+(d-b)+a")

t1=d-b
t2=c+t1
t3=t2+a


## Discusión

Suponga que hay escasez de variables temporales, así que se le ha pedido que trate de reciclarlas lo más posible, y solo crear nuevas variables ``ti`` cuando sea estrictamente necesario. Por ejemplo, para la fórmula

```
((a+b)*c)+((d-e)/f)
```

si generamos código como lo hemos hecho en los ejemplos anteriores (código de la izquierda), se usan 5 variables temporales, pero el código de la derecha muestra que se puede evaluar usando solo 2:

```
t1=a+b               t1=a+b
t2=t1+c              t1=t1*c
t3=d-e               t2=d-e
t4=t3/f              t2=t2/f
t5=t2+t4             t1=t1+t2
```

Discuta cómo se podría modificar su algoritmo para que (1) use menos variables temporales, y luego (2) para que use el mínimo posible de variables temporales.

No es necesario que implemente nada, solo que discuta cómo se podría hacer.

Para utilizar menos variables temporales se pueden reutilizar las creadas, esto se puede aplicar solo si el operador, representado  como nodo, tiene solo a uno de sus lados una expresion y al otro una hoja, ya que en ese caso se puede actualizar el valor de la variable ti por el de esta operada con la hoja. En caso contrario, como ambos lados son expresiones deben ser almacenadas y para utilizar el minimo de variables temporales una de estas puede ser reciclada actualizando su valor por el de esta operada con la otra variable temporal.
En caso de aplicar el metodo mencionado el numero de variables temporales ya no sería el numero de nodos internos (operaciones), sino que sería igual al número de nodos que poseen 2 hojas.